---
title: "LC121 - Best Time to Buy and Sell Stock"
author: "Vahram Poghosyan"
date: "2022-01-23"
categories: ["Leetcode", "Algorithms"]
image: "leetcode.png"
format:
  html:
    code-fold: true
jupyter: python3
include-after-body:
  text: |
    <script type="application/javascript" src="../../javascript/light-dark.js"></script>
---

# Problem Statement

We are given an array of `prices` where `prices[i]` is the price of a given stock on the `i`-th day.

We want to maximize our profit by choosing a single day to buy one stock and choosing a different day in the future to sell that stock. Return the maximum profit you can achieve from this transaction. If you cannot achieve any profit, return `0`.

**Example 1**

```
Input: prices = [7,1,5,3,6,4]
Output: 5
```

**Explanation** 

Buy on day 2 (price = 1) and sell on day 5 (price = 6), profit = 6-1 = 5.
Note that buying on day 2 and selling on day 1 is not allowed because you must buy before you sell

**Example 2:**

```
Input: prices = [7,6,4,3,1]
Output: 0
```

**Explanation** 

In this case, no transactions are done and the max profit = 0.

## Brute Force Solution

Consider each viable pair of days. It's easy to see that this leads to time complexity $O(n^2)$ because, for each possible day `i` that we choose to buy the stock on, there are $n-i$ possible days that we can sell the stock on. Since there are $n$ choices for which day to buy the stock on, the number of total pairs has a leading term of $n(n-1)$, so is quadratic in $n$. 

We can also think of choosing a subset of size $2$ and discarding those which have a reverse order of days. This essentially means choosing a subset of size $2$ without order (since each pair is either in the correct order or not, and we only count the one that is), so ${O \left ({n \choose 2} \right )}$ which is, of course,  $O(n^2)$.

## Non-Brute Solution

We can solve this problem in a single pass, achieving $O(n)$ complexity by a variation on [Kadane's algorithm](https://en.wikipedia.org/wiki/Maximum_subarray_problem#Kadane's_algorithm). Whereas Kadane is concerned with the contiguous subarray with maximum sum, in our case we're interested in keeping track of maximum profit. We can easily generalize Kadane's algorithm by swapping out maximum sum with our own scoring function (maximum profit). Like Kadane's, we can also prove this aglorithm's correctness using *loop invariants*. Now for the solution.

### Code

In [1]:
#| code-fold: false
def maxProfit(prices):
    min_price = float("inf") # +infinity
    max_profit = 0 

    for i in range(len(prices)):
        if prices[i] < min_price:
            min_price = prices[i]
        elif prices[i] - min_price > max_profit:
            max_profit = prices[i] - min_price

    return max_profit

prices = [7,1,5,3,6,4]
print(maxProfit(prices))

5


## Proof of Correctness

It's easy to see that, at the end of each iteration `i`, the variable `min_price` holds the lowest price dip in the stock up to, and including, the index `i`. This is called a loop invariant. Showing something is a loop invariant is a lot like proving the *inductive step* in a proof by induction (for example, when proving with recursive algorithms).

It can also be shown that `max_profit` is another loop invariant, and that it holds the maximum profit up to, and including, the index `i`. 

### Proof of Invariance

The loop does one of two things exclusively: either it updates `min_price` or it doesn't. 

Suppose it *doesn't* update `min_price` (**case 1**). The first loop invariant, `min_price` holds the lowest price dip up to, and including, the index `i`. In this case, the difference of `prices[i]` and `min_price` is then calculated and `max_profit` is updated *only* if the difference is greater than the `max_profit` at the end of the previous iteration (iteration `i-1`). This guarantees that `max_profit` holds the maximum profit up to, and including, the index `i`. 

In the other case (**case 2**), when the loop *does* update `min_price]` at iteration `i`, it enters the subsequent iteration `i+1` with `max_profit` still holding the maximum profit up to, and including, index `i` (at the beginning of the iteration). If `prices[i+1]` is, again, less than `min_price` then the loop just goes on updating `min_price` *only* until it encounters the lowest price dip (unless the price keeps dipping until the very end, in which case the proof is complete). At this iteration of the loop, let's call it `k+1`, `max_profit` still holds the maximum profit up to, and including, index `k` (because there have just been consecutive dips in price since index `i`). Since `prices[k+1]` is the lowest price dip, by assumption, in the next iteration we are necessarily in the familiar again (**case 1**). Hence, `max_profit` is a loop invariant. 

Therefore, once the loop is finished, `max_profit` will hold the maximum profit up to, and including, the last index `n`. In other words, it will hold the solution to the problem. 